Before running the next cell please add a shortcut to the shared folder at the root of your Google Drive

In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
current_path = ""

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/Academico/doctorado_programacion/doctorado/experiments/2021_01_learning_with_density_matrices

    import sys
    sys.path.append('submodules/qmc/')
    #sys.path.append('../../../../submodules/qmc/')
    print(sys.path)
    /doctorado_programacion/
else:
    import sys
    sys.path.append('submodules/qmc/')
    sys.path.append('data/')
    #sys.path.append('../../../../submodules/qmc/')
    print(sys.path)
    %cd ../../

!pwd


['/media/joag/Proyectos/Doctorado/2021_01_learning_with_density_matrices/notebooks/usps', '/media/joag/Proyectos/Doctorado/2021_01_learning_with_density_matrices', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/home/joag/.cache/pypoetry/virtualenvs/2021-01-learning-with-density-matrices-_ih_4jaw-py3.9/lib/python3.9/site-packages', '/home/joag/.cache/pypoetry/virtualenvs/2021-01-learning-with-density-matrices-_ih_4jaw-py3.9/lib/python3.9/site-packages/IPython/extensions', '/home/joag/.ipython', 'submodules/qmc/', 'data/']
/media/joag/Proyectos/Doctorado/2021_01_learning_with_density_matrices
/media/joag/Proyectos/Doctorado/2021_01_learning_with_density_matrices


In [3]:
import qmc.tf.layers as layers
import qmc.tf.models as models



In [4]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [5]:
import sklearn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold

In [6]:
# from functions.keras_wrapper_qmc import KerasClassifier

# from sklearn.model_selection import RandomizedSearchCV, KFold
# from sklearn.metrics import make_scorer

In [7]:
sys.path.append('scripts/')

In [8]:
from load_usps import load_usps

In [9]:
print(sys.path)

['/media/joag/Proyectos/Doctorado/2021_01_learning_with_density_matrices/notebooks/usps', '/media/joag/Proyectos/Doctorado/2021_01_learning_with_density_matrices', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/home/joag/.cache/pypoetry/virtualenvs/2021-01-learning-with-density-matrices-_ih_4jaw-py3.9/lib/python3.9/site-packages', '/home/joag/.cache/pypoetry/virtualenvs/2021-01-learning-with-density-matrices-_ih_4jaw-py3.9/lib/python3.9/site-packages/IPython/extensions', '/home/joag/.ipython', 'submodules/qmc/', 'data/', 'scripts/']


In [18]:
X_train, y_train, X_test, y_test = load_usps("data/usps/usps.h5")

print("shape X_train : ", X_train.shape)
print("shape y_train : ", y_train.shape)
print("shape X_test : ", X_test.shape)
print("shape y_test : ", y_test.shape)

shape X_train :  (7291, 256)
shape y_train :  (7291,)
shape X_test :  (2007, 256)
shape y_test :  (2007,)


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [20]:
from min_max_scaler import min_max_scaler
X_train, X_val, X_test = min_max_scaler(X_train, X_val, X_test)

In [21]:
gammas = [2**i for i in range(-10,10)]

In [22]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(5832, 256)
(5832,)
(1459, 256)
(1459,)
(2007, 256)
(2007,)


In [ ]:
from sklearn.kernel_approximation import RBFSampler
rbf_feature = RBFSampler(gamma=2**-5, n_components = 1000)
X_train_features = rbf_feature.fit_transform(X_train)
X_test_features = rbf_feature.transform(X_test)

from sklearn.svm import LinearSVC

model_lsvc = LinearSVC(C=2**1,tol=1e-05)
model_lsvc.fit(X_train_features, y_train)
print(model_lsvc.score(X_test_features, y_test))

In [ ]:
input_dim = X_train.shape[1]
component_dim = 1000
num_classes = np.unique(y_train).shape[0]
gamma = 2**-5
random_state=0
num_eig=0
batch_size=8
epochs = 10

In [ ]:
def create_model(input_dim, num_classes, component_dim=100, gamma=1, lr=0.01, decay=0.,
                  random_state=None, eig_percentage=0, initialize_with_rff=False,
                  type_of_rff="rff", fix_rff=False):
    '''This is a model generating function so that we can search over neural net
    parameters and architecture'''

    num_eig = round(eig_percentage * component_dim)

    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, decay=decay)

    if type_of_rff == 'rff':
        fm_x = layers.QFeatureMapRFF(input_dim, dim=component_dim, gamma=gamma, random_state=random_state)
    else:
        fm_x = layers.QFeatureMapORF(input_dim, dim=component_dim, gamma=gamma, random_state=random_state)

    if initialize_with_rff:
        qmkdc = models.QMKDClassifier(fm_x=fm_x, dim_x=component_dim, num_classes=num_classes)
        qmkdc.compile()
        qmkdc.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=0)

    qmkdc1 = models.QMKDClassifierSGD(input_dim=input_dim, dim_x=component_dim, num_eig=num_eig,
                                      num_classes=num_classes, gamma=gamma, random_state=random_state, fm_x=fm_x)

    if fix_rff:
        qmkdc1.layers[0].trainable = False

    qmkdc1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

    if initialize_with_rff:
        qmkdc1.set_rhos(qmkdc.get_rhos())

    # qmkdc1.fit(X_train, y_train_bin, epochs=epochs, batch_size=batch_size)

    return qmkdc1

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/gdrive/My Drive/logs/"

In [ ]:
# Creación del modelo sin random search. Sirve para verificar que el algoritmo este entrenando bien.
import datetime, os
logdir = os.path.join("/gdrive/My Drive/logs", "usps-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model_mnist = create_model(input_dim, num_classes, component_dim=1000, gamma=2**-6, lr=1e-06, decay=0.,
                  random_state=None, eig_percentage=0.1, initialize_with_rff=True,
                  type_of_rff="rff", fix_rff=False)

y_train_bin = tf.reshape(tf.keras.backend.one_hot(y_train, num_classes), (-1, num_classes))
y_test_bin = tf.reshape(tf.keras.backend.one_hot(y_test, num_classes), (-1, num_classes))
model_mnist.fit(X_train, y_train_bin.numpy(), epochs=12000, batch_size=32, validation_data=(X_test, y_test_bin.numpy()), 
            callbacks=[tensorboard_callback])

In [ ]:
y_test_bin = tf.reshape(tf.keras.backend.one_hot(y_test, num_classes), (-1, num_classes))
out = model_mnist.predict(X_test)
accuracy_score(y_test, out.argmax(axis=1))

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=16, verbose=1)

In [ ]:
num_classes

In [ ]:
2**30

In [ ]:
# component dimension
from scipy.stats import randint
#components_dimensions = randint(20,1500)
components_dimensions = [1000]

# gamma
#gammas = [2 ** i for i in range(-25, 25)]
gammas = [2 ** i for i in range(4, 6)]

# number of eigen values
eig_values = [1 / 10, 25 / 100, 50 / 100, 1]

# initialize with rff
#initialize_with_rff = [True, False]
initialize_with_rff = [True]

# type_of_rff_components
type_of_rff = ['rff', 'orf']

# fix_rff
fix_rff = [True]

# learning algorithm parameters
lr = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
decay = [1e-6, 1e-9, 0]

# dictionary summary
param_grid = dict(input_dim=[input_dim], num_classes=[num_classes], component_dim=components_dimensions, gamma=gammas,
                  lr=lr, decay=decay,
                  random_state=[None], eig_percentage=eig_values, initialize_with_rff=initialize_with_rff,
                  type_of_rff=type_of_rff, fix_rff=fix_rff
                  )

In [ ]:
grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid,
                              verbose=20, n_iter=1, n_jobs=1)

In [ ]:
# grid_result = grid.fit(X_train, y_train)
y_train_bin = tf.reshape(tf.keras.backend.one_hot(y_train, num_classes), (-1, num_classes)).numpy()

In [ ]:
cv_results_df = None

for i in range(25):
  grid_result = grid.fit(X_train, y_train_bin)

  if cv_results_df is None:
    cv_results_df = pd.DataFrame(grid_result.cv_results_)
  else:
    cv_results_df = pd.concat([cv_results_df, pd.DataFrame(grid_result.cv_results_)])

  cv_results_df.to_csv('qmkdc letters random search.csv')